# Photometry in Python - Photutils, DAOFIND, and Aperture Photometry

This notebook is based on a series created by Stuart Littlefair at the University of Sheffield. 
It also makes use of 'phot_helpers.py', a set of useful scripts created by Stuart Littlefair for photometry.


## Steps

At the end of this notebook you should be able to measure magnitudes for stars in your calibrated Nickel images. 

The steps are:
1. finding the stars in our image;
1. measuring instrumental magnitudes (photometry);
1. matching our stars against a sky-survey, to give us a pre-calibrated magnitude for many stars;
1. calculating the offset between instrumental and calibrated magnitudes, and applying to all stars in our image.


# Photometry with Photutils

Many tools exist to perform photometry (IRAF, GAIA, AstroImageJ etc.)
We are going to be using the Python package [photutils](http://photutils.readthedocs.org/en/latest/) to do most of the heavy lifting for us.

If you do not have photutils already installed, the code cell below will install it. Run this cell, and if there are any **errors**, ask for help (warnings are ok/expected).

In [ ]:
import sys
!{sys.executable} -m pip install photutils

In [ ]:
## if this code cell runs without error, you have successfully installed photutils!
import photutils as p

In [ ]:
# A function to perform aperture photometry
from photutils.utils import calc_total_error
from photutils.aperture import CircularAperture
from photutils.aperture import CircularAnnulus
import astropy.stats as st
import warnings
from astropy.wcs import WCS
def aperture_photometry(data, header, sources, aperture_radius, sky_inner_radius, sky_outer_radius):
    """
    Calculate Aperture Photometry on a list of sources

    Parameters
    ----------
    data:  `np.ndarray`
        A 2D array of pixel values of your data. From a FITS file, you can create this array with
        `fits.getdata`.

    header: `~astropy.fits.Header`
        A FITS Header object. From a FITS file, you can create this object with
        `fits.getheader`.

    sources: `~astropy.table.Table`
        A table of detected sources for the image. Usually, `photutils.DAOStarFinder` would be
        used to create this list.

    aperture_radius: float
        Radius of the target aperture, in pixels

    sky_inner_radius: float
        Radius of the inner aperture that makes up the sky annulus

    sky_outer_radius: float
        Radius of the outer aperture that makes up the sky annulus

    Returns
    -------
    phot_table: `~astropy.table.Table`
        A table of measurements for each source, including instrumental magnitude and error.
    """
    # make apertures around sources, and annuli for sky estimation
    positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
    apertures = CircularAperture(positions, r=aperture_radius)
    sky_annulus = CircularAnnulus(positions,
                                    r_in=sky_inner_radius, r_out=sky_outer_radius)
    annulus_masks = sky_annulus.to_mask(method='center')

    # aperture photometry - calculates total counts in apertures, with errors
    # calc_total_error uses CCD SNR equation 
    error_arr = calc_total_error(data, 12.0, egain)
    phot_table = p.aperture.aperture_photometry(data, apertures, error=error_arr)

    # calculate the Sky background. Because the sky annulus might have other
    # stars inside it, we will take a CLIPPED MEAN of the counts in the annulus
    # to try and reject the contribution from stars.
    bkg_mean = []
    for mask in annulus_masks:
        annulus_data = mask.multiply(data)
        annulus_data_1d = annulus_data[mask.data > 0]
        mean_sigclip, _, _ = st.sigma_clipped_stats(annulus_data_1d)
        bkg_mean.append(mean_sigclip)

    # now we know the mean sky counts. We multiply by ratio of annulus area to
    # target aperture area. This gives expected number of sky counts in target
    # aperture.
    bkg_mean = np.array(bkg_mean)
    phot_table['sky_mean'] = bkg_mean
    phot_table['aper_bkg'] = bkg_mean * apertures.area
    phot_table['aper_sum_bksub'] = phot_table['aperture_sum'] - phot_table['aper_bkg']

    # Tricky bit now! We want to know what Right Ascension and Declination
    # each pixel corresponds to. We will use the information in the FITS header
    # (the so-called "World Coordinate System") to work this out. 
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        wcs = WCS(header)
    ra, dec = wcs.all_pix2world(phot_table['xcenter'], phot_table['ycenter'], 0)
    phot_table['RA'] = ra
    phot_table['DEC'] = dec

    # Calculate Instrumental Magnitude and Error
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        phot_table['instrumental_mag'] = -2.5*np.log10(phot_table['aper_sum_bksub'] / header['EXPTIME'])
    phot_table['e_instrumental_mag'] = phot_table['aperture_sum_err'] / phot_table['aper_sum_bksub']

    return phot_table

In [ ]:
# We will use this function later to measure FWHM in our images
def measure_FWHM(data, sources):
    # averagely bright stars
    lims = np.percentile(sources['flux'], (55, 65))
    mask = reduce(
        np.logical_and,
        (sources['flux'] > lims[0], sources['flux'] < lims[1],
         sources['xcentroid'] > 15, sources['ycentroid'] > 15,
         sources['xcentroid'] < data.shape[1]-15, 
         sources['ycentroid'] < data.shape[0]-15
        )
    )

    positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
    tree = KDTree(positions)
    # nearest neighbours
    dist, ind = tree.query(positions, 2)

    # most isolated star of proper brightness
    idx = dist[:, 1][mask].argmax()
    location = positions[mask][idx]
    cutout = Cutout2D(data, location.T, 15)

    fig, axis = plt.subplots(nrows=1, ncols=2, figsize=(13, 6))
    norm = ImageNormalize(cutout.data, interval=AsymmetricPercentileInterval(1, 99))
    axis[0].imshow(cutout.data, cmap='Greys', origin='lower', norm=norm, interpolation='nearest')
    axis[0].set_xlabel('X')
    axis[0].set_ylabel('Y')

    xc, yc = centroid_com(cutout.data)
    x = np.arange(15) - xc
    y = np.arange(15) - yc
    X, Y = np.meshgrid(x, y)
    R = np.sqrt(X**2 + Y**2)
    axis[1].plot(R.ravel(), cutout.data.ravel(), '.')
    axis[1].set_xlabel('Distance from centre of star')
    axis[1].set_ylabel('Counts')


# Read in your image

Using the code cell below, read in one of your pre-calibrated Nickel images with plenty of stars. A good choice would be a long exposure V-band image. We're going to read the image into an array called `data` and read the header from the FITS file into a variable called `header`. 


In [ ]:
from astropy.io import fits,ascii
import numpy as np
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.visualization import AsymmetricPercentileInterval
from matplotlib import pyplot as plt
from functools import reduce
from scipy.spatial import KDTree
from astropy.nddata import Cutout2D
from photutils.centroids import centroid_com

# We need to set the gain of our CCD to allow us to measure errors correctly later
# For the Nickel, gain is 2 e/ADU
egain = 2.0
ifile = '/Users/jrees/Python/Nickel_Photometry/Data/d1068_os_bs_ff_bp_crj.fits'
data, header = fits.getdata(ifile,header=True)

# Creating a Source List

The first step is to detect our sources. We will do this using an algorithm called DAOFIND ([Stetson 1987, PASP, 99, 191](https://ui.adsabs.harvard.edu/abs/1987PASP...99..191S/abstract)). DAOFIND looks for bright regions in the image that have a peak brightness greater than some threshold and that have a size and shape similar to a Gaussian of specified FWHM.

Stars in our image will stand out above the background, and DAOFIND will find them, but we need to know what threshold to use. One way of doing this is to measure the statistics of the **background** in our image. If we measure the average value of the background, and the amount the background varies, we can look for regions that are significantly brighter than background pixels. 

Below we will do that using a "sigma-clipped" mean - this estimates the average background and the standard deviation. We then throw away all the pixels more than 3 standard deviations (sigma) away from the mean, and repeat the process. We carry on until no pixels are more than 3 standard deviations away from the average value, then calculate the mean, median and standard deviation of the remaining pixels.

You may want to tweak this value of sigma until you're happy with the sky level.

In [ ]:
# import sigma_clipping function from astropy
from astropy.stats import sigma_clipped_stats

mean_background, median_background, background_standard_deviation = sigma_clipped_stats(data, sigma=3.0)

print("The background has an average value of {:.1f} and a standard deviation of {:.1f} counts".format(
    mean_background, background_standard_deviation))

# Finding Stars

Now we know how bright our background is, and how much it varies, let's look for stars that are brighter than the background plus 5 standard deviations. That should be enough that we don't identify bright background pixels as stars by accident. The DAOFIND algorithm needs a guess for how big the stars are - as a Gaussian FWHM - we'll guess at 3 pixels for now, but you can tweak this further.

In [ ]:
from photutils.detection import DAOStarFinder

# make a star finder object to look for stars with FWHM~3 pixels that are more than 5-sigma above background
daofind = DAOStarFinder(fwhm=6.0, threshold=5*background_standard_deviation)

# Mask the data to remove the edges/corners
mask=np.ma.make_mask(data,copy=True,shrink=True,dtype=bool)
mask[:,:]=False
mask[950:,0:50]=True
mask[0:50,0:50]=True
mask[:,1010:]=True

# use it to find stars. We'll subtract the background off first, so background pixels have an average value of 0
sources = daofind(data - median_background,mask=mask)

print(sources)

The ``sources`` variable contains a table of all the detected stars. There are various columns, but the ones we are interested in is the X and Y positions of the stars, which you can find with ```sources['xcentroid']``` and ```sources['ycentroid']```.

But how do we know we've found most of the stars? Or if we are mistakenly identifying bright background pixels as stars? We can inspect our sources by-eye. To make this easier, we're going to use one of the handy functions written by Stuart Littlefair 


In [ ]:
# How big should we make the red apertures (in pixels)?
radius=7
# Set up the positions of stars (from DAOFIND) and the size of our apertures
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=radius)
# Normalise the image
norm = ImageNormalize(data, interval=AsymmetricPercentileInterval(5, 95))

fig = plt.figure(figsize=(15, 12))
apertures.plot(color='red', lw=1.5, alpha=0.5)
plt.imshow(data, cmap='Greys', origin='lower', norm=norm, interpolation='nearest')


# Tweak the detection settings

 The DAOFIND algorithm requires a threshold for star detection, and a typical FWHM of the stars in the images. Try different settings for these values, and see how they affect the detection of stars in your data. Make a decision about what values to use for this image.

# FWHM of stars in the image

We will also need an estimate for the FWHM of stars in the image. We can estimate this by-eye by plotting a bright, isolated, star and plotting the brightness against distance from the star's centre. 

Again, a handy tool exists to do this already. Using the plot below, figure out your FWHM in pixels. Using the Nickel plate scale you can convert it to arcseconds (you do remember the value right?)

In [ ]:

measure_FWHM(data, sources)

# Aperture Photometry

So, we have a list of detected sources and positions in the image, and an idea of the FWHM of stars in the image. Now we can perform aperture photometry on all of these sources. 
In brief the function performs the following steps:

1. Add up the counts from each source within a target aperture
2. Measure the sky brightness around each source using the sky annulus
3. Subtract the sky contribution from the counts in step 1.
4. Calculate instrumental magnitude from the counts and exposure time.


# Perform Aperture Photometry

You'll need to pick values for the aperture radii. 

Target apertures want to be big enough to accept a decent fraction of the flux, but not so large that the measurements are very noisy, or contaminated by nearby stars. As a rule of thumb this aperture might have a radius of 1.5-2x the FWHM.

Sky Annuli want to be wide enough to accurately measure the sky, but not so large that the annuli overlap nearby stars.

To get started, try values around 5, 10, 20 pixels for these apertures. We will tweak them later on.

If you want to view extra information about what 'aperture_photometry' is doing, you can do:

help(aperture_photometry)

In [ ]:
Vmag_inst = aperture_photometry(data, header, sources, 15, 25, 35)


# Photometric Calibration

Now we have a table of instrumental magnitudes (and much besides) in it. The next step is photometric calibration. As a reminder, this involves finding the average value of the *difference* between the calibrated and instrumental magnitudes for all our stars.

Since we have instrumental magnitudes and sky positions (RA, Dec) for a number of stars, we must find the matching stars in an online catalog of calibrated magnitudes. We will use the [APASS](https://www.aavso.org/apass) catalog; a catalog which combines several other sky surveys to provide data in many filters across much of the sky. Crucially, in this case it includes B, V, r, and i magnitudes, the filters used for our photometry.

To perform the cross-matching we will use the [astroquery](https://astroquery.readthedocs.io/en/latest/) Python library, and we have our aperture photometry results in a an astropy table.

In [ ]:
from astropy import units as u
from astroquery.xmatch import XMatch

Our photometry is stored as an astropy table, which is a handy object for reading and writing tabular data. These astropy tables play nicely with Jupyter notebooks, so you can simply type the name of the table in a code cell to see the table displayed in the browser.

In [ ]:
Vmag_inst

In [ ]:
from astropy.table import Table
from astroquery.astrometry_net import AstrometryNet

ast = AstrometryNet()
#ast.api_key = '##########'
# To avoid broadcasting it to the world, the Astrometry.net API key is read in from a file
f = open("/Users/jrees/Python/Nickel_Photometry/astromnet.dat", "r")
ast.api_key = f.readline()
f.close()

#sources = Table.read('catalog.fits')
# Sort sources in ascending order
sources.sort('flux')
# Reverse to get descending order
sources.reverse()

image_width = 1024
image_height = 1024
wcs_header = ast.solve_from_source_list(sources['xcentroid'], sources['ycentroid'],
                                        image_width, image_height,
                                        solve_timeout=120)



In [ ]:
w = WCS(wcs_header)
Vmag_inst['RA'], Vmag_inst['DEC'] = w.wcs_pix2world(Vmag_inst['xcenter'],Vmag_inst['ycenter'],1)


In [ ]:
from astropy.coordinates import SkyCoord
# Create a SkyCoord object for your first filter

#change stan coords depending on standard star 
StanRA=237.9996
StanDEC=32.94861

coo_v = SkyCoord(Vmag_inst['RA'], Vmag_inst['DEC'], unit=u.deg)
coo=SkyCoord(StanRA,StanDEC,unit=u.deg)

#change standard Vmag depending on standard star 
Stan_Vmag=10.73
d2d = coo.separation(coo_v)
zeropoint_v= Stan_Vmag-Vmag_inst[np.where(d2d == d2d.min())]['instrumental_mag']
print(zeropoint_v[0])

The important columns for our uses are the measured centres of our stars in RA and Dec (**RA** and **DEC**) and the instrumental magnitude and uncertainty (**instrumental_mag** and **e_instrumental_mag**). Note that the magnitude uncertainty is calculated using the CCD signal-to-noise equation.

# Calculate the zero-point

> Calculate the median value of ```delta_mag```. **If you have evidence for saturated stars**, you can use a NumPy *mask* to only calculate the median of stars that are not saturated 

> The median value of ```delta_mag``` is our estimate of $-kX + m_{\rm zp}$ - i.e the value we want to add to our instrumental magnitudes to get a calibrated magnitude. You can add this value to the **instrumental_mag** column of the VMag_inst table easily using ```VMag_inst['calibrated_mag'] = VMag_inst['instrumental_mag'] + zp```, where ```zp``` is the median value you found.

In [ ]:
# Now apply the zeropoint to the instrumental magnitudes
Vmag_inst['calibrated_mag'] = Vmag_inst['instrumental_mag'] + zeropoint_v[0]

In [ ]:
import math
exptime = 10
sky_brightness = zeropoint_v[0] - 2.5*math.log10((mean_background/exptime)/(0.36*0.36))
print (sky_brightness)

---------
# Other filters and making a CMD

If we repeat the above steps for other images, we can measure our stellar magnitudes in all of our other filters

# The B-band data

 Assuming that you started with V, repeat the steps for your B-band data. Make a plot of the difference between instrumental B-band magnitude and APASS B-band magnitude. Calculate the offset needed to correct your instrumental mags (zeropoint).

 Finally, add your zeropoint to the **instrumental_mag** column of the B-band table to make a new **calibrated_mag** column.

In [ ]:
# Grab the data
ifile = '/Users/jrees/Python/Nickel_Photometry/Data/d1067_os_bs_ff_bp_crj.fits'
data, header = fits.getdata(ifile,header=True)

# Measure the background
mean_background, median_background, background_standard_deviation = sigma_clipped_stats(data, sigma=3.0)

print("The background has an average value of {:.1f} and a standard deviation of {:.1f} counts".format(
    mean_background, background_standard_deviation))

# Find the stars 
daofind = DAOStarFinder(fwhm=6.0, threshold=5*background_standard_deviation)
sources = daofind(data - median_background)
print(sources)

# Check everything looks good
radius=6
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=radius)
norm = ImageNormalize(data, interval=AsymmetricPercentileInterval(5, 95))
fig = plt.figure(figsize=(15, 12))
apertures.plot(color='red', lw=1.5, alpha=0.5)
plt.imshow(data, cmap='Greys', origin='lower', norm=norm, interpolation='nearest')

# Take a look at the FWHM in this image
measure_FWHM(data, sources)

# Time for photometry
Bmag_inst = aperture_photometry(data, header, sources, 15, 25, 35)



# Crossmatching our calibrated magnitudes

Now you should have a two astropy tables: One with calibrated V-band magnitudes and one with calibrated B-band magnitudes. We need to cross-match these tables with each other, to find which stars in the V-band table match with stars in the B-band table. We can't use ```astroquery```'s Xmatch for this, since they are both local tables. Instead, we will use the ```SkyCoord``` object from astropy, which is meant to work with coordinates on the sky. 



In [ ]:
# Create a SkyCoord object for your first filter
coo_v2 = SkyCoord(Vmag_inst['RA'], Vmag_inst['DEC'], unit=u.deg)

# And for the second filter 
coo_b2 = SkyCoord(Bmag_inst['RA'], Bmag_inst['DEC'], unit=u.deg)


# match every entry in coo_v with the nearest entry in coo_b to identify the same stars in each catalogue
idx, distance_2d, distance_3d = coo_v2.match_to_catalog_sky(coo_b2)

# using idx as a slice for the Bmag_inst table will sort it so that Bmag_inst[0] is the closest match to Vmag_inst[0]
Bmag_inst = Bmag_inst[idx]

# And finally we have to ensure the closest match is actually useful. 
# Some of the 'closest matches' might actually be very far away. 
dist_mask = distance_2d < 3 * u.arcsec
Bmag_inst = Bmag_inst[dist_mask]
Vmag_inst = Vmag_inst[dist_mask]

# Plotting a CMD 

 If we want to plot a CMD from our data, we can extract the calibrated magnitudes, calculate ```B-V``` and plot a colour-magnitude diagram of ```B-V``` against ```V```. Your plot should look something like the one below, which is for the cluster NGC 7789.

<img src="./images/CMD.png" style="margin: 0px" width=750px/>


In [ ]:
B = Bmag_inst['calibrated_mag']
V = Vmag_inst['calibrated_mag']

plt.figure(figsize=(20,12))
plt.plot(B-V,V, 'bo')
plt.xlabel('B-V')
plt.ylabel('V Mag')
plt.gca().invert_yaxis()
plt.show

# Other filters

If you have data in other filters (and time!) you can go ahead and repeat the above steps for the other filters.


In [ ]:
# YOUR CODE HERE
#R
# Grab the data
ifile = '/Users/jrees/Python/Nickel_Photometry/Data/d1069_os_bs_ff_bp_crj.fits'
data, header = fits.getdata(ifile,header=True)

# Measure the background
mean_background, median_background, background_standard_deviation = sigma_clipped_stats(data, sigma=3.0)

print("The background has an average value of {:.1f} and a standard deviation of {:.1f} counts".format(
    mean_background, background_standard_deviation))

# Find the stars 
daofind = DAOStarFinder(fwhm=6.0, threshold=5*background_standard_deviation)
sources = daofind(data - median_background)
print(sources)

# Check everything looks good
radius=6
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=radius)
norm = ImageNormalize(data, interval=AsymmetricPercentileInterval(5, 95))
fig = plt.figure(figsize=(15, 12))
apertures.plot(color='red', lw=1.5, alpha=0.5)
plt.imshow(data, cmap='Greys', origin='lower', norm=norm, interpolation='nearest')

# Take a look at the FWHM in this image
measure_FWHM(data, sources)

# Time for photometry
Rmag_inst = aperture_photometry(data, header, sources, 15, 25, 35)

Rmag_inst['RA'], Rmag_inst['DEC'] = w.wcs_pix2world(Rmag_inst['xcenter'],Rmag_inst['ycenter'],1)

coo_r = SkyCoord(Rmag_inst['RA'], Rmag_inst['DEC'], unit=u.deg)
coo=SkyCoord(StanRA,StanDEC,unit=u.deg)

#change standard Vmag depending on standard star 
Stan_Rmag=10.884
d2d = coo.separation(coo_r)
zeropoint_r= Stan_Rmag-Rmag_inst[np.where(d2d == d2d.min())]['instrumental_mag']
print(zeropoint_r[0])

exptime = 10
sky_brightness = zeropoint_r[0] - 2.5*math.log10((mean_background/exptime)/(0.18*0.18))
print ("Sky Brightness = ", sky_brightness)

# And apply the zeropoint
Rmag_inst['calibrated_mag'] = Rmag_inst['instrumental_mag'] + zeropoint_r



In [ ]:
# YOUR CODE HERE
#R
# Grab the data
ifile = '/Users/jrees/Python/Nickel_Photometry/Data/d1070_os_bs_ff_bp_crj.fits'
data, header = fits.getdata(ifile,header=True)

# Measure the background
mean_background, median_background, background_standard_deviation = sigma_clipped_stats(data, sigma=3.0)

print("The background has an average value of {:.1f} and a standard deviation of {:.1f} counts".format(
    mean_background, background_standard_deviation))

# Find the stars 
daofind = DAOStarFinder(fwhm=6.0, threshold=5*background_standard_deviation)
sources = daofind(data - median_background)
#print(sources)

# Check everything looks good
radius=6
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=radius)
norm = ImageNormalize(data, interval=AsymmetricPercentileInterval(5, 95))
fig = plt.figure(figsize=(15, 12))
apertures.plot(color='red', lw=1.5, alpha=0.5)
plt.imshow(data, cmap='Greys', origin='lower', norm=norm, interpolation='nearest')

# Take a look at the FWHM in this image
#measure_FWHM(data, sources)

# Time for photometry
Imag_inst = aperture_photometry(data, header, sources, 15, 25, 35)

Imag_inst['RA'], Imag_inst['DEC'] = w.wcs_pix2world(Imag_inst['xcenter'],Imag_inst['ycenter'],1)

coo_i = SkyCoord(Imag_inst['RA'], Imag_inst['DEC'], unit=u.deg)
coo=SkyCoord(StanRA,StanDEC,unit=u.deg)

#change standard Vmag depending on standard star 
Stan_Imag=10.961
d2d = coo.separation(coo_i)
zeropoint_i= Stan_Imag-Imag_inst[np.where(d2d == d2d.min())]['instrumental_mag']
print(zeropoint[0])

exptime = 10
sky_brightness = zeropoint_i[0] - 2.5*math.log10((mean_background/exptime)/(0.18*0.18))
print ("Sky Brightness = ", sky_brightness)

# And apply the zeropoint
Imag_inst['calibrated_mag'] = Imag_inst['instrumental_mag'] + zeropoint_i


In [ ]:
# And now let's do some clusters
# V
ifile = '/Users/jrees/Python/Nickel_Photometry/Data/d1124_os_bs_ff_bp_crj.fits'
data, header = fits.getdata(ifile,header=True)

# Measure the background
mean_background, median_background, background_standard_deviation = sigma_clipped_stats(data, sigma=3.0)

print("The background has an average value of {:.1f} and a standard deviation of {:.1f} counts".format(
    mean_background, background_standard_deviation))

# Find the stars 
daofind = DAOStarFinder(fwhm=6.0, threshold=5*background_standard_deviation)
sources = daofind(data - median_background)
#print(sources)

# Check everything looks good
radius=6
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=radius)
norm = ImageNormalize(data, interval=AsymmetricPercentileInterval(5, 95))
fig = plt.figure(figsize=(15, 12))
apertures.plot(color='red', lw=1.5, alpha=0.5)
plt.imshow(data, cmap='Greys', origin='lower', norm=norm, interpolation='nearest')

# Take a look at the FWHM in this image
#measure_FWHM(data, sources)

# Time for photometry
Vmag_inst = aperture_photometry(data, header, sources, 15, 25, 35)


exptime = 100
sky_brightness = zeropoint_v[0] - 2.5*math.log10((mean_background/exptime)/(0.18*0.18))
print ("Sky Brightness = ", sky_brightness)

# And apply the zeropoint
Vmag_inst['calibrated_mag'] = Vmag_inst['instrumental_mag'] + zeropoint_v


# B
ifile = '/Users/jrees/Python/Nickel_Photometry/Data/d1120_os_bs_ff_bp_crj.fits'
data, header = fits.getdata(ifile,header=True)

# Measure the background
mean_background, median_background, background_standard_deviation = sigma_clipped_stats(data, sigma=3.0)

print("The background has an average value of {:.1f} and a standard deviation of {:.1f} counts".format(
    mean_background, background_standard_deviation))

# Find the stars 
daofind = DAOStarFinder(fwhm=6.0, threshold=5*background_standard_deviation)
sources = daofind(data - median_background)
#print(sources)

# Check everything looks good
radius=6
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=radius)
norm = ImageNormalize(data, interval=AsymmetricPercentileInterval(5, 95))
fig = plt.figure(figsize=(15, 12))
apertures.plot(color='red', lw=1.5, alpha=0.5)
plt.imshow(data, cmap='Greys', origin='lower', norm=norm, interpolation='nearest')

# Take a look at the FWHM in this image
#measure_FWHM(data, sources)

# Time for photometry
Bmag_inst = aperture_photometry(data, header, sources, 15, 25, 35)


exptime = 100
sky_brightness = zeropoint_b[0] - 2.5*math.log10((mean_background/exptime)/(0.18*0.18))
print ("Sky Brightness = ", sky_brightness)

# And apply the zeropoint
Bmag_inst['calibrated_mag'] = Bmag_inst['instrumental_mag'] + zeropoint_b

In [ ]:
# Create a SkyCoord object for your first filter
coo_v2 = SkyCoord(Vmag_inst['RA'], Vmag_inst['DEC'], unit=u.deg)

# And for the second filter 
coo_b2 = SkyCoord(Bmag_inst['RA'], Bmag_inst['DEC'], unit=u.deg)


# match every entry in coo_v with the nearest entry in coo_b to identify the same stars in each catalogue
idx, distance_2d, distance_3d = coo_v2.match_to_catalog_sky(coo_b2)

# using idx as a slice for the Bmag_inst table will sort it so that Bmag_inst[0] is the closest match to Vmag_inst[0]
Bmag_inst = Bmag_inst[idx]

# And finally we have to ensure the closest match is actually useful. 
# Some of the 'closest matches' might actually be very far away. 
dist_mask = distance_2d < 3 * u.arcsec
Bmag_inst = Bmag_inst[dist_mask]
Vmag_inst = Vmag_inst[dist_mask]

B = Bmag_inst['calibrated_mag']
V = Vmag_inst['calibrated_mag']

plt.figure(figsize=(20,12))
plt.plot(B-V,V, 'bo')
plt.xlabel('B-V')
plt.ylabel('V Mag')
plt.gca().invert_yaxis()
plt.show